In [ ]:
import os
from copy import deepcopy

import numpy as np

import pydicom
from pymedphys.type import DeliveryData
from pymedphys.dcm import anonymise_dicom
from pymedphys.coll import find_relevant_control_points

In [ ]:
from pymedphys.logfile import delivery_data_from_logfile

In [ ]:
data_root = r'C:\Users\sbiggs\logfile2dicom\endometrium'
from glob import glob

In [ ]:
logfile_path = glob(os.path.join(
    data_root, 'logfile', '*'
))[0]

logfile_path

In [ ]:
original = glob(os.path.join(
    data_root, 'original', '*'
))[0]

plan_name = os.path.basename(original)

In [ ]:
filepath_out = glob(os.path.join(
    data_root, 'dicom_240ms', plan_name
))[0]

filepath_out

In [ ]:
# filepath = '../../tests/data/dcmplan/TinyVMAT/TinyFS_TinyVMAT.dcm'
# filepath_out = '../../tests/data/dcmplan/test/test.dcm'
dcm = pydicom.read_file(original, force=True)

In [ ]:
# logfile_path = os.path.join(
#     '..', '..',
#     'tests', 'data', 'trf', 'elekta_reference',
#     '17_04_24 02_23_05 Z 1-1_1ARCVMAT.trf'
# )
# logfile_path

In [ ]:
delivery_data = delivery_data_from_logfile(logfile_path)

In [ ]:
def strip_delivery_data(delivery_data, skip_size):
    new_delivery_data = []
    for item in delivery_data:
        new_delivery_data.append(np.array(item)[::skip_size].tolist())
        
    return DeliveryData(*new_delivery_data)

delivery_data = strip_delivery_data(delivery_data, 6)  # make 240 ms res instead of 40 ms

In [ ]:
def filter_out_irrelivant_control_points(delivery_data):
    
    relvant_control_points = find_relevant_control_points(
        delivery_data.monitor_units)
    
    new_delivery_data = []
    for item in delivery_data:
        new_delivery_data.append(np.array(item)[relvant_control_points].tolist())
        
    return DeliveryData(*new_delivery_data)


delivery_data = filter_out_irrelivant_control_points(delivery_data)

In [ ]:

# delivery_data.gantry[0:first_change]

In [ ]:
def extract_one_gantry_angle(delivery_data, gantry_angle):
    near_angle = np.abs(np.array(delivery_data.gantry) - gantry_angle) < 3
    assert np.all(np.diff(np.where(near_angle)[0]) == 1)
    
    new_delivery_data = []
    for item in delivery_data:
        new_delivery_data.append(np.array(item)[near_angle].tolist())
        
    new_delivery_data[0] = np.round(np.array(new_delivery_data[0]) - new_delivery_data[0][0], decimals=7).tolist()
        
    return DeliveryData(*new_delivery_data)


delivery_data = extract_one_gantry_angle(delivery_data, -120)

In [ ]:
# delivery_data

In [ ]:
# delivery_data.gantry

In [ ]:
# delivery_data.gantry

In [ ]:
len(delivery_data.monitor_units)

In [ ]:
np.shape(np.array(delivery_data.mlc))
mlc = np.array(delivery_data.mlc)

np.shape(mlc[:,:,0])

In [ ]:
def convert_mlc_format(mlc):
    
    dicom_mlc_format = []
    for control_point in mlc:
        concatenated = np.hstack([-control_point[-1::-1,1], control_point[-1::-1,0]])
#         concatenated[-5::] = 1.8
        dicom_mlc_format.append(concatenated.astype(str).tolist())
        
    return dicom_mlc_format
        
converted_mlc = convert_mlc_format(mlc)

# converted_mlc[0]

In [ ]:
def convert_jaw_format(jaw):
    dicom_jaw_format = []
    for control_point in jaw:
        control_point[0]

In [ ]:
new_jaw = np.array(delivery_data.jaw)
new_jaw[:,1] = -new_jaw[:,1]

converted_jaw = new_jaw.astype(str)
converted_jaw[:,1] = new_jaw.astype(str)[:,0]
converted_jaw[:,0] = new_jaw.astype(str)[:,1]
converted_jaw = converted_jaw.tolist()

In [ ]:
# converted_jaw

In [ ]:
def movement_check(angles):
    float_angles = angles.astype(np.float64)
    float_angles[float_angles >= 180] = float_angles[float_angles >= 180] - 360
    diff = np.append(np.diff(float_angles), 0)
    
    movement = (np.empty_like(angles)).astype(str)
    movement[diff > 0] = 'CW'
    movement[diff < 0] = 'CC'
    movement[diff == 0] = 'NONE'
    
    return movement

In [ ]:
# delivery_data.gantry

In [ ]:
diff = np.append(np.diff(delivery_data.gantry), 0)
movement = (np.empty_like(delivery_data.gantry)).astype(str)

movement[diff > 0] = 'CW'
movement[diff < 0] = 'CC'
movement[diff == 0] = 'NONE'

# movement[:] = 'CW'
# movement[-1] = 'NONE'

In [ ]:
# movement.tolist()

In [ ]:
converted_gantry = np.array(delivery_data.gantry)
converted_gantry[converted_gantry<0] = converted_gantry[converted_gantry<0] + 360

converted_gantry = converted_gantry.astype(str).tolist()
# converted_gantry

In [ ]:
# converted_mlc[0]

In [ ]:
monitor_units = delivery_data.monitor_units

In [ ]:
# skip_step = 1

# monitor_units_trim = np.concatenate(
#     [np.array(delivery_data.monitor_units)[::skip_step],
#      np.array(delivery_data.monitor_units)[-1::]]
# ).tolist()
# converted_mlc_trim = np.concatenate([
#     converted_mlc[::skip_step], converted_mlc[-1::]
# ]).tolist()


# # monitor_units_trim

In [ ]:
# monitor_units_trim[-1]

In [ ]:
# delivery_data.monitor_units[-1]

In [ ]:
control_point_sequence = dcm.BeamSequence[0].ControlPointSequence

In [ ]:
init_cp = deepcopy(control_point_sequence[0])
subsequent_cp = deepcopy(control_point_sequence[-1])

init_cp.GantryAngle = converted_gantry[0]
init_cp.GantryRotationDirection = movement[0]
init_cp.BeamLimitingDevicePositionSequence[0].LeafJawPositions = converted_jaw[0]

init_cp.BeamLimitingDevicePositionSequence[1].LeafJawPositions = converted_mlc[0]
# init_cp.BeamLimitingDevicePositionSequence[1].LeafJawPositions = dummy_mlc_pos_string

# subsequent_cp.GantryAngle = "90.0"
# subsequent_cp.BeamLimitingDevicePositionSequence[0].LeafJawPositions = ['-130.0', '130.0']

remaining_cps = []
for i, (mu, mlc_cp, jaw_cp, move_cp, gantry_cp) in enumerate(zip(monitor_units[1::], 
                                     converted_mlc[1::],
                                     converted_jaw[1::],
                                     movement[1::],
                                     converted_gantry[1::])):    
    current_cp = deepcopy(subsequent_cp)
    current_cp.ControlPointIndex = str(i+1)
    current_cp.GantryAngle = gantry_cp
    current_cp.GantryRotationDirection = move_cp
    current_cp.BeamLimitingDevicePositionSequence[0].LeafJawPositions = jaw_cp
    current_cp.BeamLimitingDevicePositionSequence[1].LeafJawPositions = mlc_cp
    current_cp.CumulativeMetersetWeight = np.around(
        mu / delivery_data.monitor_units[-1], decimals=5)
    
    remaining_cps.append(current_cp)
    

all_control_points = [init_cp] + remaining_cps

In [ ]:
# dcm = pydicom.read_file(filepath, force=True)
# dcm.save_as(filepath_out)

In [ ]:
# edited_dcm = deepcopy(dcm)

In [ ]:
# dcm = pydicom.read_file(filepath, force=True)

# edited_dcm = deepcopy(dcm)

# # first_cp = edited_dcm.BeamSequence[0].ControlPointSequence[0]
# # last_cp = edited_dcm.BeamSequence[0].ControlPointSequence[-1]

# # last_cp.ControlPointIndex = "1"
# # last_cp.GantryAngle = "200.0"


# # # edited_dcm.FractionGroupSequence[0].ReferencedBeamSequence[0].BeamMeterset = (
# # #     str(monitor_units_trim[-1]))


# # edited_dcm.BeamSequence[0].ControlPointSequence = [
# #     first_cp, last_cp
# # ]
# # edited_dcm.BeamSequence[0].NumberOfControlPoints = "2"

# edited_dcm.save_as(filepath_out)
# edited_dcm

In [ ]:
# edited_dcm

In [ ]:
# remaining_cps

In [ ]:
# len(edited_dcm.BeamSequence)

In [ ]:
# edited_dcm.BeamSequence[-2]

In [ ]:
# edited_dcm.FractionGroupSequence[0].ReferencedBeamSequence

In [ ]:
edited_dcm = deepcopy(dcm)


edited_dcm.FractionGroupSequence[0].ReferencedBeamSequence[-2].BeamMeterset = (
    str(monitor_units[-1]))

edited_dcm.FractionGroupSequence[0].ReferencedBeamSequence = [
    edited_dcm.FractionGroupSequence[0].ReferencedBeamSequence[-2]
]

edited_dcm.BeamSequence[-2].ControlPointSequence = all_control_points
edited_dcm.BeamSequence[-2].NumberOfControlPoints = len(all_control_points)

edited_dcm.BeamSequence = [
    edited_dcm.BeamSequence[-2]
]

edited_dcm.save_as(filepath_out)

In [ ]:
# edited_dcm